# Remodl AI SDK

## Overview and ground truths
1. Atomic use of AI and LLMS as functional code enables flexibility
2. AI functions execute.
3. Agents are higher level and complex. They are variants of a ReAct agentic approach
4. Agents can work with other agents (leveraging a2a)
5. Specialized agents can create new agents
6. An end user may experience what feels like a single agent, but in fact it is a team of agents working together
7. All Remodl SDK code should be composable, declarative, and able to be "assembled" (like lego blocks)
8. If an agent or AI function can't be comprehended in natural language, then it isn't a single agent.
9. We assign special meaning to specific words:
    - Will: defines a required parameter or action
    - Should: defines a suggested parameter or action
    - May: defines an optional parameter or action
    - Can (alt "Is able to"): defines a capability
    - Must (alt "Is required", "needs to", "needs"): defines a constraint
    - Is: defines a state
    - Will: defines a required action
    - Has: defines a property
    - Uses: defines a tool


## Basic Config

We use a config method very close to litellm's original one, just prepopulating internally certain defaults, like the API_BASE_URL

In [2]:
 !pip install pydantic typing-extensions typing


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from remodl import Remodl
from pydantic import BaseModel, Field
import os
from typing import Optional, List, Dict, An
from typing_extensions import Annotated, 
remodl_client = Remodl.auth(
    api_key="remodl-api-key",
    project_name="my-project",
    project_create_if_missing=True,
)

rc = remodl_client.configure(model="chat-small", temperature=0.7, max_tokens=4096)

# We also allow for other models to be used, if available:
remodl_custom = remodl_client.configure(model="openai/gpt-4o-mini", temperature=0.7, max_tokens=4096)



rc.Activate()
# we basically extend the litellm config to include remodl specific fields
class RemodlConfig(BaseModel):
    model_family: str | 'remodl'
    model_name: str
    model: str = Field(default_factory=lambda self: f"{self.model_family}/{self.model_name}")
    temperature: float = 0.7
    max_tokens: int = 4096
    api_key: str | os.getenv("REMODL_API_KEY")
    project_name: str = Field(default_factory=lambda self: os.getenv("REMODL_PROJECT_NAME") | "default")
    project_create_if_missing: bool = True
    api_base_url: str = "https://api.remodl.com/v1"


class Remodl(RemodlConfig):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client = Remodl.auth(
            api_key=self.api_key,
            project_name=self.project_name,
            project_create_if_missing=self.project_create_if_missing
        )

    def auth(self, api_key: str, project_name: str, project_create_if_missing: bool = True):
        self.api_key = api_key
        self.project_name = project_name
        self.project_create_if_missing = project_create_if_missing
        
        # we need some "login" function to validate agains the remodl api
        # Mock authentication call to REMODL_GATEWAY
        import requests
        
        gateway_url = os.getenv("REMODL_GATEWAY", "http://localhost")
        gateway_port = os.getenv("REMODL_PORT", "4000")
        auth_endpoint = f"{gateway_url}:{gateway_port}/authenticate"
        
        try:
            response = requests.post(auth_endpoint, json={
                "api_key": api_key,
                "project_name": project_name,
                "project_create_if_missing": project_create_if_missing
            })
            
            if response.status_code == 200:
                auth_data = response.json()
                print(f"Authentication successful for project: {project_name}")
                return auth_data
            else:
                raise Exception(f"Authentication failed: {response.status_code} - {response.text}")
                
        except requests.exceptions.RequestException as e:
            print(f"Warning: Could not connect to Remodl gateway at {auth_endpoint}")
            print(f"Error: {e}")
            print("Proceeding with mock authentication...")
            return {"status": "mock_auth", "project": project_name, "api_key": api_key[:8] + "..."}

    def configure(self, model: str, temperature: float = 0.7, max_tokens: int = 4096):
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        #TODO: add a configure hook call to the remodl api
        return self

    def Activate(self):
        #TODO: add an activate hook call to the remodl api
        return self

    #...

    







## Function as prompt
We have employ a few sentinels to allow for easy, functional non-agentic calls to the llm, offloading efforts

`@ai`: an ai function that calls an llm. Normally these would leverage smaller remodl-provided models
`@with_profile`: activates a given profile

under the hood we are mapping these to simple DSPY signatures

In [ ]:
from remodl.functions import ai, _ai

@ai
def refine_question(question: str, context: str) -> str:
    details = {
        "instructions": "Refine a question based on the context.",
        "expects": [
            {"name": "question", "type": "str", "required": True, "description": "The question to refine"},
            {"name": "context", "type": "str", "required": False, "description": "The context to use for the refinement"}
        ],
        "returns": [
            {"name": "refined_question", "type": "str", "required": True, "description": "The refined question"}
        ]
    }
refine_question(question="What is the capital of France?", context=context")
    # this maps to a DSPY signature like:
    
    def templatize_ai_function():
        """Generate a DSPy signature class from function details."""
        import dspy
        from typing import get_type_hints
        
        # Build signature field definitions
        signature_parts = []
        
        # Process input fields
        for field in details["expects"]:
            field_name = field['name']
            field_type = field['type']
            field_desc = field['description']
            
            # Handle optional default values
            if 'default' in field:
                default_clause = f"default={field['default']}"
            else:
                default_clause = ""
            
            field_def = f"{field_name}: {field_type} = dspy.InputField({default_clause}, description='{field_desc}')"
            signature_parts.append(field_def)
        
        # Process output fields
        for field in details["returns"]:
            field_name = field['name']
            field_type = field['type']
            field_desc = field['description']
            
            field_def = f"{field_name}: {field_type} = dspy.OutputField(description='{field_desc}')"
            signature_parts.append(field_def)
        
        # Generate the signature class code
        signature_code = f'''
class GeneratedSignature(dspy.Signature):
    """{details["instructions"]}"""
    
    {chr(10).join("    " + part for part in signature_parts)}
"""
        
        # Execute the code to create the class
        exec_globals = {'dspy': dspy}
        exec(signature_code, exec_globals)
        
        return exec_globals['GeneratedSignature']




In [36]:
%%writefile dspy_signature_template.py


import dspy
student_lm=dspy.LM(
        "openrouter/mistralai/codestral-2508",
        api_key="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",
       
    )

teacher_lm=dspy.LM(
    "openrouter/qwen/qwen3-235b-a22b-thinking-2507",
    api_key="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",
)

from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any

details = {
    "instructions": "Refine a question based on the context.",
    "expects": [
        {
            "name": "question", 
            "type": "str", 
            "required": True, 
            "description": "The question to refine"
        },
        {
            "name": "context", 
            "type": "str", 
            "required": False, 
            "description": "The context to use for the refinement"
        }
    ],
    "returns": [
        {
            "name": "refined_question", 
            "type": "str", 
            "required": True, 
            "description": "The refined question"
        }
    ]
}

details_natural = {
    "name": "RefineQuestion",
    "instructions": """
You will expect a question. 
You may be provided context, but it is not required. 
Return the refined question, and nothing else.

Context is a json object with the following fields:
- history: a list of messages between the user and the assistant
- knowledge: an aray of strings, ending in :: and then a unique identified called a ukid, which looks like 'h.dsds.sdsd/sdsd' or 'hdsdsds.*'
- user_info: a json object with the following fields, all of which are optional:
"""
}

class FieldSpec(BaseModel):
    """Specification for a single field in a DSPy signature."""
    name: str = Field(description="The field name")
    type: str = Field(description="The Python type (e.g., 'str', 'int', 'list[str]')")
    description: Optional[str] = Field(default=None, description="Optional description of the field")
    default: Optional[Any] = Field(default=None, description="Optional default value for input fields")


class SignatureSpec(BaseModel):
    """Complete specification for a DSPy signature."""
    name: str = Field(description="The class name for the signature")
    instructions: str = Field(description="The docstring instructions for the signature")
    input_fields: List[FieldSpec] = Field(description="List of input field specifications")
    output_fields: List[FieldSpec] = Field(description="List of output field specifications")


signature_template = """
CORRECT DSPy SIGNATURE FORMAT:

class ClassName(dspy.Signature):
    \\\"\\\"\\\"Brief task description.\\\"\\\"\\\"
    
    field_name: type = dspy.InputField()
    field_name: type = dspy.InputField(desc="optional description")
    field_name: type = dspy.OutputField()
    field_name: type = dspy.OutputField(desc="optional description")

CRITICAL RULES:
1. Must inherit from 'dspy.Signature' (never just 'Signature')
2. Docstring is brief and describes the task
3. Use 'desc' parameter, NOT 'description'
4. NO comments like "# Input fields" or "# Output fields"
5. Type annotation BEFORE the = sign
6. Format: field_name: type = dspy.InputField(desc="...")

CORRECT EXAMPLES:

Example 1 - Simple:
class EmotionClassifier(dspy.Signature):
    \\\"\\\"\\\"Classify the emotion expressed in a sentence.\\\"\\\"\\\"
    
    sentence: str = dspy.InputField()
    emotion: str = dspy.OutputField(desc="One of: sadness, joy, love, anger, fear, surprise")

Example 2 - With descriptions:
class CheckCitationFaithfulness(dspy.Signature):
    \\\"\\\"\\\"Verify that the text is based on the provided context.\\\"\\\"\\\"
    
    context: str = dspy.InputField(desc="facts here are assumed to be true")
    text: str = dspy.InputField()
    faithfulness: bool = dspy.OutputField()

Example 3 - Multiple inputs/outputs:
class AnswerQuestion(dspy.Signature):
    \\\"\\\"\\\"Answer questions with short factoid answers.\\\"\\\"\\\"
    
    context: list[str] = dspy.InputField(desc="may contain relevant facts")
    question: str = dspy.InputField()
    answer: str = dspy.OutputField(desc="often between 1 and 5 words")

WRONG FORMAT (DO NOT GENERATE):
class Bad(dspy.Signature):
    # Input fields  ← NO COMMENTS
    field: str = dspy.InputField(description="...")  ← USE 'desc' NOT 'description'
"""


keyword_semantics = """
SEMANTIC KEYWORDS IN USER DESCRIPTIONS:
- Will: required parameter → InputField() with no default
- Should: suggested parameter → InputField() with default
- May: optional parameter → InputField() with default=None
- Will return: required output → OutputField()
- Must/Can/Is/Has/Uses: goes in docstring instructions
"""


class SignatureGenerator(dspy.Signature):
    """Generate valid DSPy signature code from natural language task descriptions.
    
    Parse semantic keywords to determine inputs and outputs. Follow the exact
    DSPy signature format from the template. Output must be valid Python code.
    
    TEMPLATE TO FOLLOW:
    {template}
    
    SEMANTIC KEYWORDS:
    {keywords}
    """.format(template=signature_template, keywords=keyword_semantics)
    
    task_description: str = dspy.InputField(desc="Natural language task description using semantic keywords")
    signature_name: str = dspy.InputField(desc="Class name for the signature (e.g., 'RefineQuestion')")
    signature_code: str = dspy.OutputField(desc="Valid Python code wrapped in ```python blocks, following DSPy format exactly")


# Initialize the generator
generate_signature = dspy.ChainOfThought(SignatureGenerator)


# Example: User writes this (no DSPy knowledge)
user_task = """
I need to refine user questions to be clearer.

You will receive a question string.
You may receive context containing history and knowledge snippets.

You must preserve the original intent.
You can improve clarity and specificity.

You will return the refined question as a string.
"""

# Expected output format:
expected_output = """
```python
class RefineQuestion(dspy.Signature):
    \\\"\\\"\\\"Refine user questions to be clearer while preserving intent.\\\"\\\"\\\"
    
    question: str = dspy.InputField()
    context: dict = dspy.InputField(desc="optional history and knowledge snippets")
    refined_question: str = dspy.OutputField()
```
"""
# Initialize the signature generator
generate_signature = dspy.ChainOfThought(SignatureGenerator)


# Example: Using semantic keywords
user_task_description = """
I need to refine user questions.

You will receive a question string.
You may receive context with these properties:
- history: previous conversation messages
- knowledge: relevant information (ends with :: and a UKID)
- user_info: optional user data

You must preserve the original intent.
You can improve clarity and specificity.
You should make questions more suitable for search.

You will return the refined question as a string.
"""

create_signature = generate_signature(task_description=user_task_description, signature_name="RefineQuestion")
print(create_signature.signature_code)


Overwriting dspy_signature_template.py


In [ ]:
"""
Train SignatureGenerator to convert natural language → DSPy signatures

Uses semantic keyword parsing to generate proper DSPy signature code
from natural language task descriptions.
"""

import dspy
import pandas as pd
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, MIPROv2
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any

# Load training data
df = pd.read_csv("dspy_signature_training_dataset.csv")
print(f"Loaded {len(df)} training examples")

# Configure DSPy
dspy.configure(
    lm=dspy.LM("openai/gpt-4o-mini"),  # Student model
    adapter=dspy.JSONAdapter()
)

# Teacher model for optimization
teacher_lm = dspy.LM("openai/gpt-4o")

# Define the signature template and keywords (from dspy_signature_template.py)
signature_template = """
CORRECT DSPy SIGNATURE FORMAT:

class ClassName(dspy.Signature):
    \\\"\\\"\\\"Brief task description.\\\"\\\"\\\"
    
    field_name: type = dspy.InputField()
    field_name: type = dspy.InputField(desc="optional description")
    field_name: type = dspy.OutputField()
    field_name: type = dspy.OutputField(desc="optional description")

CRITICAL RULES:
1. Must inherit from 'dspy.Signature' (never just 'Signature')
2. Docstring is brief and describes the task
3. Use 'desc' parameter, NOT 'description'
4. NO comments like "# Input fields" or "# Output fields"
5. Type annotation BEFORE the = sign
6. Format: field_name: type = dspy.InputField(desc="...")

CORRECT EXAMPLES:

Example 1 - Simple:
class EmotionClassifier(dspy.Signature):
    \\\"\\\"\\\"Classify the emotion expressed in a sentence.\\\"\\\"\\\"
    
    sentence: str = dspy.InputField()
    emotion: str = dspy.OutputField(desc="One of: sadness, joy, love, anger, fear, surprise")

Example 2 - With descriptions:
class CheckCitationFaithfulness(dspy.Signature):
    \\\"\\\"\\\"Verify that the text is based on the provided context.\\\"\\\"\\\"
    
    context: str = dspy.InputField(desc="facts here are assumed to be true")
    text: str = dspy.InputField()
    faithfulness: bool = dspy.OutputField()
"""

keyword_semantics = """
SEMANTIC KEYWORDS IN USER DESCRIPTIONS:
- Will: required parameter → InputField() with no default
- Should: suggested parameter → InputField() with default
- May: optional parameter → InputField() with default=None
- Will return: required output → OutputField()
- Must/Can/Is/Has/Uses: goes in docstring instructions
"""

# Define the SignatureGenerator signature
class SignatureGenerator(dspy.Signature):
    """Generate valid DSPy signature code from natural language task descriptions.
    
    Parse semantic keywords to determine inputs and outputs. Follow the exact
    DSPy signature format from the template. Output must be valid Python code.
    
    TEMPLATE TO FOLLOW:
    {template}
    
    SEMANTIC KEYWORDS:
    {keywords}
    """.format(template=signature_template, keywords=keyword_semantics)
    
    task_description: str = dspy.InputField(desc="Natural language task description using semantic keywords")
    signature_code: str = dspy.OutputField(desc="Valid Python code following DSPy format exactly")

# Create the program
class SignatureGeneratorProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate = dspy.ChainOfThought(SignatureGenerator)
    
    def forward(self, task_description):
        return self.generate(task_description=task_description)

# Prepare training data
trainset = []
for _, row in df.iterrows():
    example = dspy.Example(
        task_description=row['natural_language'],
        signature_code=row['code']
    ).with_inputs('task_description')
    trainset.append(example)

# Split train/dev
split_point = int(len(trainset) * 0.8)
train = trainset[:split_point]
dev = trainset[split_point:]

print(f"\nTrain: {len(train)} examples")
print(f"Dev: {len(dev)} examples")

# Define evaluation metric
def validate_dspy_signature(gold, pred, trace=None):
    """
    Validate that generated code is a proper DSPy Signature.
    
    Checks:
    1. Inherits from dspy.Signature (not just 'Signature')
    2. Has docstring
    3. Uses dspy.InputField() and dspy.OutputField()
    4. Uses 'desc' parameter (not 'description')
    5. No comments like "# Input fields"
    6. Proper type annotations (field: type = ...)
    7. Code is syntactically valid Python
    """
    if not pred or not hasattr(pred, 'signature_code'):
        return 0.0
    
    pred_code = pred.signature_code
    
    # Remove code fences if present
    if '```python' in pred_code:
        try:
            pred_code = pred_code.split('```python')[1].split('```')[0].strip()
        except IndexError:
            return 0.0
    
    score = 0.0
    errors = []
    
    # CRITICAL: Must inherit from dspy.Signature (not just Signature)
    if 'class' in pred_code and '(dspy.Signature)' in pred_code:
        score += 0.2
    else:
        errors.append("Missing 'class X(dspy.Signature)'")
        return 0.0  # Fail fast
    
    # CRITICAL: Must have docstring
    if '"""' in pred_code:
        score += 0.1
    else:
        errors.append("Missing docstring")
    
    # CRITICAL: Must have at least one InputField
    if 'dspy.InputField(' in pred_code:
        score += 0.15
    else:
        errors.append("No dspy.InputField() found")
        return score * 0.5  # Partial credit
    
    # CRITICAL: Must have at least one OutputField
    if 'dspy.OutputField(' in pred_code:
        score += 0.15
    else:
        errors.append("No dspy.OutputField() found")
        return score * 0.5  # Partial credit
    
    # RULE: Must use 'desc' not 'description'
    if 'description=' in pred_code and 'dspy.InputField' in pred_code or 'dspy.OutputField' in pred_code:
        errors.append("Uses 'description=' instead of 'desc='")
        score -= 0.1
    
    # RULE: Should not have comments like "# Input fields"
    if '# Input' in pred_code or '# Output' in pred_code:
        errors.append("Contains comments like '# Input fields'")
        score -= 0.05
    
    # VALIDATION: Try to parse as Python
    try:
        compile(pred_code, '<string>', 'exec')
        score += 0.2  # Valid Python syntax
    except SyntaxError as e:
        errors.append(f"Syntax error: {e}")
        score -= 0.2
    
    # VALIDATION: Check type annotation format (field: type = ...)
    import re
    # Look for pattern: word: word = dspy.
    if re.search(r'\w+:\s*\w+\s*=\s*dspy\.(Input|Output)Field', pred_code):
        score += 0.1
    else:
        errors.append("Incorrect type annotation format")
    
    # BONUS: Try to actually execute and instantiate
    try:
        exec_globals = {'dspy': dspy}
        exec(pred_code, exec_globals)
        # Find the class that was defined
        sig_class = None
        for name, obj in exec_globals.items():
            if isinstance(obj, type) and issubclass(obj, dspy.Signature) and obj != dspy.Signature:
                sig_class = obj
                break
        
        if sig_class:
            score += 0.1  # Successfully created signature class
            
            # Try to instantiate
            try:
                instance = sig_class()
                score += 0.05  # Can instantiate
            except Exception:
                pass
    except Exception as e:
        errors.append(f"Cannot execute: {e}")
    
    if errors and trace:
        print(f"Validation errors: {errors}")
    
    return max(0.0, min(score, 1.0))  # Clamp between 0 and 1

# Create baseline program
print("\n" + "="*60)
print("Creating baseline program...")
print("="*60)
program = SignatureGeneratorProgram()

# Evaluate baseline
print("\nEvaluating baseline...")
baseline_eval = Evaluate(
    devset=dev,
    metric=validate_dspy_signature,
    num_threads=1,
    display_progress=True,
    display_table=5
)
baseline_score = baseline_eval(program)
print(f"\n📊 Baseline Score: {baseline_score:.2%}")

# Optimize with BootstrapFewShot
print("\n" + "="*60)
print("Optimizing with BootstrapFewShot...")
print("="*60)

optimizer = BootstrapFewShot(
    metric=validate_dspy_signature,
    max_bootstrapped_demos=4,
    max_labeled_demos=8,
    max_rounds=1,
    teacher_settings=dict(lm=teacher_lm)
)

optimized_program = optimizer.compile(
    student=program,
    trainset=train
)

# Evaluate optimized program
print("\nEvaluating optimized program...")
eval_optimized = Evaluate(
    devset=dev,
    metric=validate_dspy_signature,
    num_threads=1,
    display_progress=True,
    display_table=5
)
optimized_score = eval_optimized(optimized_program)

# Results
print("\n" + "="*60)
print("RESULTS")
print("="*60)
print(f"Baseline Score:  {baseline_score:.2%}")
print(f"Optimized Score: {optimized_score:.2%}")
print(f"Improvement:     {(optimized_score - baseline_score):.2%}")

# Save the optimized program
save_path = "./signature_generator_optimized.json"
optimized_program.save(save_path)
print(f"\n✅ Optimized program saved to: {save_path}")

# Test on a new example
print("\n" + "="*60)
print("TESTING ON NEW EXAMPLE")
print("="*60)

test_description = """
I need to analyze customer feedback.

You will receive feedback text.
You will receive the product category.
You may receive previous feedback for context.

You must identify the main sentiment.
You can extract specific issues mentioned.
You should rate urgency.

You will return the sentiment (positive/negative/neutral).
You will return a list of issues found.
You will return an urgency score from 1 to 5.
"""

print("\nInput:")
print(test_description)
print("\nGenerated Signature:")

result = optimized_program(task_description=test_description)
print(result.signature_code)

# Calculate cost
if hasattr(dspy.settings, 'lm') and hasattr(dspy.settings.lm, 'history'):
    total_calls = len(dspy.settings.lm.history)
    print(f"\n💰 Total LLM calls: {total_calls}")

print("\n✅ Training complete!")



In [ ]:
import remodl
from remodl.agents import Agent,  Skill, Team, Workflow, State
from remodl.agents.role import Role, expects, returns, capabilities
from remodl.agents.a2a import A2A
from remodl.skills import Skill, Task, Rule,  Learn
from remodl.workflows import Step, Workflow, Decision, Exception
from remodl.helpers import aiFunction, _aiFunction
from typing import List, Dict, Any, Optional
from remodl.tools import Tool, ToolResult, ToolError, MCPTools
from remodl.lexiq import Lexiq, Memory, Knowledge
from remodl.profiles import Profile, with_profile, _with_profile


"""
We stay relatively close to the litellm interface for configure, as its simple.  We have certain defaults that we use.

remodl.configure -> litellm.configure. By default we have certain integrations that we use.
For models - we alias the remodl/remodl-chat-small to something effecient, like a GPT-OSS-20B, but it doesn't matter 

"""
remodl_chat = remodl.configure(model="remodl/remodl-chat-small", api_key="remodl-api-key", temperature=0.7, max_tokens=4096)
# We also allow for other models to be used, if available:
remodl_custom = remodl.configure(model="openai/gpt-4o-mini", api_key="remodl-api-key", temperature=0.7, max_tokens=4096)

# Additionally, we enable the use of "Profiles" - which are a way to configure the model for a specific task.
# We have certain defaults that we use, but we also allow for other models to be used, if available:
chat_profile = Profile(
    name="Chat",
    model="remodl/remodl-chat-small",
    temperature=0.7,
    max_tokens=4096,
    use_when = "When you need to chat with a user"
)

manager_profile = Profile(
    name="Manager",
    model="remodl/remodl-chat-large",
    temperature=0.7,
    max_tokens=4096,
    use_when = "When you need to manage a team"
)

# Profiles allow for easy switching if someone doesn't want to juggle multiple defined variable, 
#usage leverages a sentinel '@with_profile'

@with_profile(chat_profile) # or @with_profile("Chat")

def do_something():
    return "Hello, world!"


"""
We work on the following conceptual truths:
1. Atomic use of AI and LLMS as functional code enables flexibility
2. AI functions execute.
3. Agents are higher level and complex. They are variants of a ReAct agentic approach
4. Agents can work with other agents (leveraging a2a)
5. Specialized agents can create new agents
6. An end user may experience what feels like a single agent, but in fact it is a team of agents working together
7. All Remodl SDK code should be composable, declarative, and able to be "assembled" (like lego blocks)
8. If an agent or AI function can't be comprehended in natural language, then it isn't a single agent.
9. We assign special meaning to specific words:
    - Will: defines a required parameter or action
    - Should: defines a suggested parameter or action
    - May: defines an optional parameter or action
    - Can (alt "Is able to"): defines a capability
    - Must (alt "Is required", "needs to", "needs"): defines a constraint
    - Is: defines a state
    - Will: defines a required action
    - Has: defines a property
    - Uses: defines a tool


"""



In [ ]:

research_agent_role = Role(name="Research Agent"):
    """
    You are a research assistant that can help with tasks and questions.
    """
    question: str = expects(description="The question to answer")
    context: Optional[str] = expects(description="The context to use for the answer")
    knowledge: Optional[Knowledge] = expects(description="The knowledge to use for the answer")
    can_learn: bool = returns(description="Whether the agent can learn from the question")
    



context7 = MCPTool(
    name="Context",
    use_remodl=True,
)
tools = [context7]

research_agent = Agent(
    name="Research Agent",
    description="Research Agent is a research assistant that can help with tasks and questions.",
    role=research_agent_role,
    tools=tools,
)

research_agent.modelCard(enable_a2a=True):
        # A2A Model Card Configuration
        a2a_config = {
            "model_name": "research-agent-v1.0",
            "model_type": "agent-to-agent",
            "capabilities": [
                "question_answering",
                "research_assistance", 
                "context_analysis",
                "knowledge_retrieval"
            ],
            "input_schema": {
                "question": {"type": "string", "required": True},
                "context": {"type": "string", "required": False},
                "knowledge": {"type": "object", "required": False}
            },
            "output_schema": {
                "answer": {"type": "string", "required": True},
                "confidence": {"type": "float", "range": [0.0, 1.0]},
                "sources": {"type": "array", "items": {"type": "string"}},
                "can_learn": {"type": "boolean", "required": True}
            },
            "performance_metrics": {
                "accuracy": 0.92,
                "response_time_ms": 150,
                "knowledge_coverage": 0.85
            },
            "limitations": [
                "Requires internet access for real-time research",
                "May have knowledge cutoff limitations",
                "Context window limited to 32k tokens"
            ],
            "training_data": {
                "sources": ["academic_papers", "web_content", "knowledge_bases"],
                "last_updated": "2024-01-15",
                "size": "10M interactions"
            },
            "ethical_considerations": {
                "bias_mitigation": True,
                "privacy_preserving": True,
                "fact_checking": True
            }
        }




        name="My Role",
        description="My Role is a helpful assistant that can help with tasks and questions."
agent 